In [ ]:
# !pip install pyfim
# !pip install pandas
# !pip install matplotlibx 
# !pip install unlzw
# !pip install pyfim

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)
import Orange
from sklearn.model_selection import train_test_split
import pandas as pd
from Orange.data.pandas_compat import table_from_frame,table_to_frame


from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
from yarc import CBA #unable to import it 
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

In [ ]:
def runCBA(X,y,target):
    X = discretiseRule(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    txns = TransactionDB.from_DataFrame(train, target=target) #../../
    txnstest = TransactionDB.from_DataFrame(test, target=target) #../../

    cba = CBA()
    cba.fit(txns)
    cba.target_class
    cba.pre.rules
    
    cbaTrainAcc=cba.rule_model_accuracy(txns)
    # get the best association rules

    accuracy=cba.rule_model_accuracy(txnstest) #Part 2 model
    # get the best association rules
    rules = top_rules(txns.string_representation)
    # convert them to class association rules
    cars = CARlist(rules)
    predictor = M2Classi(cars, txns).build()

    print("CBA Train Accuracy:",str(cbaTrainAcc))
    print("CBA Accuracy:", accuracy)

# Iris

In [ ]:
iris = pd.read_csv ('iris.csv') #discretised iris from their database #currently its the pure one without discretising
iris = iris.dropna()
X=iris[["sepallength","petalwidth","sepalwidth","petallength"]]
y=iris[["class"]]
runCBA(X,y,"class")

# Wave

In [ ]:
wave = pd.read_csv ('wave.csv')
wave = wave.dropna()
X=wave[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
y=wave[["class"]]
runCBA(X,y,"class")

# Heart

In [ ]:
heart = pd.read_csv ('heart.csv')
heart = heart.dropna()
X=heart[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=heart[["class"]]
runCBA(X,y,"class")

# Breast Cancer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
breastCancer = pd.read_csv ('breastCancer.csv')
breastCancer = breastCancer.dropna()
X=breastCancer[["menopause", "age", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
#encoding the normal data
for col in ["menopause","node-caps","breast","breast-quad","irradiat"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["age","tumor-size","inv-nodes"]
for i in ordinalData:
   columns = X[i].unique()
   columns.sort() #sorting the labels
   for j in range(len(columns)):
      X.loc[X[i]==columns[j],i] = j
y=breastCancer[["class"]]
runCBA(X,y,"class")

# German

In [ ]:
german = pd.read_csv ('german.csv')
german = german.dropna()
X=german[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
for col in ['checkAccStatus','credHist','purpose','savAccBond','emplySince','personalStatSex','otherDebtGuar','prpty','otherInstallPlans','housing','job','telephone','frgnWorker']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=german[["goodBad"]]
runCBA(X,y,"goodBad")

# University Student Placement

In [ ]:
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv")
campusPlacement.head()
campusPlacement = campusPlacement.drop(['salary'],1)
campusPlacement = campusPlacement.dropna()
X = campusPlacement[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p']]
#creating a function to encode categorical features into numerical
for col in ['gender','ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=campusPlacement[['status']]
runCBA(X,y,"status")

In [ ]:
stroke = pd.read_csv('healthcare-dataset-stroke-data.csv')
stroke = stroke.drop(['id'],1)
stroke = stroke.dropna()
X=stroke[["gender", "age", "heart_disease", "ever_married", "work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
#encoding the normal data
for col in ["gender","ever_married","work_type","Residence_type", "smoking_status"]:
   X[col] = LabelEncoder().fit_transform(X[col])
y=stroke[["stroke"]]
runCBA(X,y,"stroke")